<a href="https://colab.research.google.com/github/tanongsakintean/google_colab/blob/main/tanongsakintean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install bs4

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=afad7d122ffa279417f027a442f1bc94528628ad56c7c04bdb507191516bc4dc
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [ ]:
pip install requests

In [ ]:
pip install pandas

In [ ]:
pip install openpyxl

In [ ]:
pip install azure-storage-blob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import bs4
import requests
import openpyxl

In [ ]:
item = 1
name_list = []
price_list = []

while item <= 10:
  data = requests.get('https://girlsnationactive.com/collections/all?page=1')
  soup = bs4.BeautifulSoup(data.text)

  name = soup.find_all('a',{'class':'boost-pfs-filter-product-item-title'})
  price = soup.find_all('p',{'class':'boost-pfs-filter-product-item-price'})

  for i in name:
    name_list.append(i.text)

  for i in price:
    price_list.append(i.text)


  print('Complete item number: ',item)
  item += 1

table = pd.DataFrame([name_list,price_list]).transpose()
table.columns = ['name','price']
table.set_index('name')
table.to_excel('market.xlsx',engine='openpyxl') #file is created on google colab cloud

#https://girlsnationactive.com/collections/all?page=1
from azure.storage.blob import BlobServiceClient

#ไปที่ storage account แล้วเลือก access key เพื่อ copy key , account และ conection string ใส่ในช่องว่างแต่ละตัวแปร

storage_account_key = "NzS73tQ1CMmPUowS3+v4tohlDMHMEnIPlKBrjowfDKCZR2H0yjbzBDTTziQwM8plee/wjNWg5bE2+AStpJ8sTg=="
storage_account_name = "bigdatalab3"
connection_string = "DefaultEndpointsProtocol=https;AccountName=bigdatalab3;AccountKey=NzS73tQ1CMmPUowS3+v4tohlDMHMEnIPlKBrjowfDKCZR2H0yjbzBDTTziQwM8plee/wjNWg5bE2+AStpJ8sTg==;EndpointSuffix=core.windows.net"
container_name = "blobinsearchdata"

#upload file to blobstorage

def uploadToBlobStorage(file_path,file_name):
  blob_service_client = BlobServiceClient.from_connection_string(connection_string)
  blob_client = blob_service_client.get_blob_client(container=container_name,blob=file_name)
  try:
    if blob_client.exists():
        # Delete blob
        blob_client.delete_blob()
        print("Blob deleted successfully!")
    else:
        print("Blob does not exist!")
    with open(file_path,"rb") as data:
      blob_client.upload_blob(data)
    print(f"Uploaded {file_name}.successfully!")
  except Exception as e:
    print("Failed to upload files to container. Error:" + str(e))

uploadToBlobStorage('/content/market.xlsx','market.xlsx') #ไฟล์ที่อ่านจาก google drive เพื่อส่งไปยังไฟล์ใน container





Complete item number:  1
Complete item number:  2
Complete item number:  3
Complete item number:  4
Complete item number:  5
Complete item number:  6
Complete item number:  7
Complete item number:  8
Complete item number:  9
Complete item number:  10
Blob does not exist!
Uploaded market.xlsx.successfully!


In [ ]:
# download_blobs_parallel.py
# Python program to bulk download blobs from azure storage
# Uses latest python SDK() for Azure blob storage
# Requires python 3.6 or above
import pandas as pd
import os
from multiprocessing.pool import ThreadPool
from azure.storage.blob import BlobServiceClient, BlobClient
from azure.storage.blob import ContentSettings, ContainerClient

# IMPORTANT: Replace connection string with your storage account connection string
# Usually starts with DefaultEndpointsProtocol=https;...
MY_CONNECTION_STRING ="DefaultEndpointsProtocol=https;AccountName=bigdatalab3;AccountKey=NzS73tQ1CMmPUowS3+v4tohlDMHMEnIPlKBrjowfDKCZR2H0yjbzBDTTziQwM8plee/wjNWg5bE2+AStpJ8sTg==;EndpointSuffix=core.windows.net"

# Replace with blob container name
MY_BLOB_CONTAINER = "blobinsearchdata"

# Replace with the local folder where you want downloaded files to be stored
LOCAL_BLOB_PATH = "/content/download_data"


#self คือ ทุกครั้งที่รัน self มันจะไปเรียกรันทุกอันให้

class AzureBlobFileDownloader:
  def __init__(self):
    print("Intializing AzureBlobFileDownloader")

    # Initialize the connection to Azure storage account
    self.blob_service_client =  BlobServiceClient.from_connection_string(MY_CONNECTION_STRING)
    self.my_container = self.blob_service_client.get_container_client(MY_BLOB_CONTAINER)

  def download_all_blobs_in_container(self):
        # get a list of blobs
        my_blobs = self.my_container.list_blobs()
        result = self.run(my_blobs)
        for filename in result:
            file_path = os.path.join(LOCAL_BLOB_PATH, filename)
            with open(file_path, "rb") as file:
                content = file.read()
                print(f"Content of {filename}:\n{content}\n")
                # Use pandas to read the data
                data_df = pd.read_excel(content)

                # Customize display options
                pd.set_option("display.max_columns", None)  # Display all columns
                pd.set_option("display.expand_frame_repr", False)  # Disable wrapping
                pd.set_option("display.precision", 2)  # Set decimal precision

                # Display the pandas DataFrame
                print(data_df)

  def run(self,blobs):
    # Download 10 files at a time!
    with ThreadPool(processes=int(10)) as pool:
     return pool.map(self.save_blob_locally, blobs)

  def save_blob_locally(self,blob):
    file_name = blob.name
    print(file_name)
    bytes = self.my_container.get_blob_client(blob).download_blob().readall()

    # Get full path to the file
    download_file_path = os.path.join(LOCAL_BLOB_PATH, file_name)
    # for nested blobs, create local path as well!
    os.makedirs(os.path.dirname(download_file_path), exist_ok=True)

    with open(download_file_path, "wb") as file:
      file.write(bytes)
    return file_name

# Initialize class and upload files
azure_blob_file_downloader = AzureBlobFileDownloader()
azure_blob_file_downloader.download_all_blobs_in_container()

Intializing AzureBlobFileDownloader
market.xlsx
Content of market.xlsx:
b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xe4n\nW\x07AMb\x81\x00\x00\x00\xb1\x00\x00\x00\x10\x00\x00\x00docProps/app.xmlM\x8e=\x0b\x021\x10D\xff\xcaq\xbd\xb7A\xc1Bb@\xd0R\xb0\xb2\x0f{\x1b/\x90dC\xb2B~\xbe9\xc1\x8fn\x1eo\x18F\xdf\ng*\xe2\xa9\x0e-\x86T\x8f\xe3"\x92\x0f\x00\x15\x17\x8a\xb6N]\xa7n\x1c\x97h\xa5cy\x00;\xe7\x91\xce\x8c\xcfHI`\xab\xd4\x1e\xa8\t\xa5\x99\xe6M\xfe\x0e\x8eF\x9fr\x0e\x1e\xadxN\xe6\xea\xb1pe\'\xc3\xa5!\x05\r\xffrm\xde\xa9\xd45\xef&\xf5\x96\x1f\xd6\xf0;i^PK\x03\x04\x14\x00\x00\x00\x08\x00\xe4n\nWR\xebE_\xee\x00\x00\x00+\x02\x00\x00\x11\x00\x00\x00docProps/core.xml\xcd\x92\xc1j\xc30\x0c\x86_e\xf8\x9e\xc8\x89Y\x19&\xf5\xa5\xa3\xa7\x0e\x06+l\xecfl\xb55\x8bcck$}\xfb%Y\x9b2\xb6\x07\xd8\xd1\xd2\xefO\x9f@\x8d\x89\xd2\x84\x84\xcf)DL\xe40\xdf\r\xbe\xed\xb24q\xcdNDQ\x02dsB\xafs9&\xba\xb1y\x08\xc9k\x1a\x9f\xe9\x08Q\x9b\x0f}D\xa89_\x81G\xd2V\x93\x86\tX\xc4\x85\xc8Tc\x8d4\t5\x85t\xc1[\xb3\xe0\xe3gjg\x985\x80-z\xec